# Emerging Technologies QA with pdf data

* Date: Dec 03, 2023
* Author: Dawan Savage Bell
* W0465310
* Mileston 10



### Resources used 
https://medium.com/@alinaeem_57283/building-a-pdf-querying-bot-with-langchain-edb71ebedcc

In [27]:
!pip install pulsar-client
!pip install chromadb

ERROR: Could not find a version that satisfies the requirement pulsar-client (from versions: none)
ERROR: No matching distribution found for pulsar-client


  Using cached chromadb-0.4.18-py3-none-any.whl.metadata (7.4 kB)
  Using cached chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.104.1-py3-none-any.whl.metadata (24 kB)
  Using cached uvicorn-0.24.0.post1-py3-none-any.whl.metadata (6.4 kB)
  Using cached posthog-3.1.0-py2.py3-none-any.whl.metadata (2.0 kB)
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
  Using cached chromadb-0.4.17-py3-none-any.whl.metadata (7.3 kB)
  Using cached chromadb-0.4.16-py3-none-any.whl.metadata (7.3 kB)
  Using cached chromadb-0.4.15-py3-none-any.whl.metadata (7.2 kB)
  Usin

ERROR: Cannot install chromadb==0.1.0, chromadb==0.2.0, chromadb==0.3.0, chromadb==0.3.1, chromadb==0.3.2, chromadb==0.3.25, chromadb==0.3.26, chromadb==0.3.27, chromadb==0.3.29, chromadb==0.4.0, chromadb==0.4.1, chromadb==0.4.10, chromadb==0.4.11, chromadb==0.4.12, chromadb==0.4.13, chromadb==0.4.14, chromadb==0.4.15, chromadb==0.4.16, chromadb==0.4.17, chromadb==0.4.18, chromadb==0.4.2, chromadb==0.4.3, chromadb==0.4.4, chromadb==0.4.5, chromadb==0.4.6, chromadb==0.4.7, chromadb==0.4.8 and chromadb==0.4.9 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [7]:
# !pip install unstructured
# !pip install pypdf
# !pip install openai
# !pip install langchain
# !pip install pdf2image
# !pip install tiktoken
# !pip install lark
#!pip install pinecone-client
!pip install chromadb


  Using cached unstructured-0.11.2-py3-none-any.whl.metadata (25 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 330.3 kB/s eta 0:00:05
     - -------------------------------------- 0.0/1.5 MB 326.8 kB/s eta 0:00:05
     --- ------------------------------------ 0.1/1.5 MB 853.3 kB/s eta 0:00:02
     -------- ------------------------------- 0.3/1.5 MB 1.5 MB/s eta 0:00:01
     ----------- ---------------------------- 0.5/1.5 MB 1.8 MB/s eta 0:00:01
     ---------------- ----------------------- 0.6/1.5 MB 2.0 MB/s eta 0:00:01
     --------------------- ------------------ 0.8/1.5 MB 2.4 MB/s eta 0:00:01
     --------------------------- ------------ 1.0/1.5 MB 2.6 MB/s eta 0:00:01
     -------------------------------

In [12]:
try:
    import os
    import json
    import PyPDF2
    import pinecone
    
    from tqdm.autonotebook import tqdm
    from langchain import HuggingFaceHub
    from langchain.document_loaders import PyPDFLoader
    from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, TokenTextSplitter
    
    # selecting the embedding 
    from langchain.embeddings import HuggingFaceEmbeddings
    from langchain.embeddings.openai import OpenAIEmbeddings
    
    from langchain.vectorstores import Chroma, Pinecone
    # expose the index in a retriever obj/instance
    from langchain.chains import RetrievalQA
    
    print("imports completed")

except Exception as e:
    print("Error: {}".format(e))

imports completed


In [9]:
try:
    with open("./keys/keys.json") as f:
        tokens = json.load(f)
        open_token = tokens["open_ai_token"]
        pinecone_token = tokens["pine_cone_token"]
        pinecone_env = tokens["pine_cone_api_env"]
        os.environ['OPENAI_API_KEY'] = open_token
        os.environ['PINECONE_API_KEY'] = pinecone_token
        print("environment set")
except Exception as e:
    print(e)
    print("token not set")    

environment set


# Reading in the PDF

In [32]:
pdf_loader = PyPDFLoader("./docs/albert_einstein_relativity.pdf")
page_data = pdf_loader.load()

In [33]:
page_data[1].page_content
page_data[1].metadata

{'source': './docs/albert_einstein_relativity.pdf', 'page': 1}

In [28]:
print(f'There are {len(page_data)} document(s) in the data')
print(f'There are {len(page_data[0].page_content)} characters in the document')

There are 184 document(s) in the data
There are 266 characters in the document


# Splitting the PDF
* splitting by tokens due to LLM's having token limits.
* When splitting by chunks, we are also counting the number of tokens. 
* splitting by characters passed in, chunk size measured by 'tiktoken' tokenizer

In [7]:
with open("./docs/albert_einstein_relativity.pdf") as f:
    #
    relativity_pdf = PyPDF2.PdfReader("./docs/albert_einstein_relativity.pdf")
    # creating empty list to store the text from each page
    text = ""
    # iterating through each page in the pdf
    for page in relativity_pdf.pages:
        # extracting the text from each page
        text += page.extract_text()

# split the doc into 
# chunk size is the number of characters
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=20)

# split the text into chunks
chunks = text_splitter.split_text(text)

In [8]:
print(chunks[1])

 the 
simplest and most intelligible form, and on the 
 
1 The mathematical fundaments of the special theory of rela-
tivity are to be found in the original papers of H. A. Lorentz, A. 
Einstein, H. Minkowski,* published under the title Das Relativitäts- 
prinzip (The Principle of Relativity) in B. G. Teubner’s collection 
of monographs Fortschritte der mathematischen Wissenschaften (Ad-
vances in the Mathematical Sciences), also in M. Laue’s exhaustive 
book Das Relativitäts prinzip — published by Friedr. Vieweg & Son, 
Braunschweig. The general theory of relativity, together with the 
necessary parts of the theory of invariants, is dealt with in the 
author’s book Die Grundlagen der allgemeinen Relativitätstheorie 
(The Foundations of the General Theory of Relativity) — Joh. 
Ambr. Barth, 1916; this book assumes some familiarity with the 
special theory of relativity. 
 
v 
 
[* Minkowski‘ — J.M.] T vi                               RELATIVITY 
 
whole, in the sequence and connection 

In [17]:
chunks[1].metadata

AttributeError: 'str' object has no attribute 'metadata'

# Creating Vector Database using Pinecone
* creates a local instance of a vector database after creating embeddings of the pdf file

In [24]:
# create embeddings
embeddings = OpenAIEmbeddings()

# create a vector store
vector_store = Chroma.from_texts(chunks, embeddings)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [23]:
# create the vector store
vector_store = Pinecone.from_texts(chunks, embeddings, index_name=index_name)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# Creating LLM based off of bloom model from Huggingface

* from BigScience Workshop. 
* architecture of Bloom similar to GPT3, trained on 46 different languages and 13 programming languages.
* https://huggingface.co/docs/transformers/model_doc/bloom

In [5]:
# get the BLOOm Model from HuggingFace
llm = HuggingFaceHub(repo_id="bigscience/bloom",
                     model_kwargs={"temperature":0.1,
                                   "max_length":100})

c:\Users\Savag\.conda\envs\nscc_env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


# Text Splitter
* simple splitting the text by each character.
* https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/character_text_splitter

In [6]:
# loading in document
pdf_path = "./docs/albert_einstein_relativity.pdf"
loader = PyPDFLoader(pdf_path)
doc = loader.load()

# split the doc into 
# chunk size is the number of characters
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(doc)

NameError: name 'PyPDFLoader' is not defined

# Embeddings
* translating the chunk of texts to series of numbers (embeddings)
* using a pre-trained model for generating the embeddings
* resource:["https://pub.aimind.so/llm-embeddings-explained-simply-f7536d3d0e4b"]

In [24]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

# Vector DB
* https://python.langchain.com/docs/modules/data_connection/vectorstores/
* resource:["https://www.pinecone.io/learn/vector-database/"]

In [31]:
# create the vectorestore to use index
db = Chroma.from_documents(texts, embeddings)

In [64]:

try:
    retriever = db.as_retriever(search_type="similarity",
                                search_kwargs={"k": 3} 
                                )
except Exception as e:
    print("error: ", e)
    
    
try:
    # Initialize RetrievalQA with the required parameters
    # Note: return_scores is removed as it's not a valid parameter
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
except Exception as e:
    # Print any error that occurs during the initialization
    print("error: ", e)

In [69]:
try:
    # ask a question
    question = "what is relativity?"
    answers = qa({"query":question})

    # print the type and value of answers
    # print(type(answers))
    # print(answers.keys())
    print(answers["query"])
    
    print(answers["result"])    
    
    #print(answers)
except Exception as e:
    # Print any error that occurs during the execution
    print("An error occurred: ", e)

An error occurred:  (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 38563f94-80e1-4725-bb77-d4482160f515)')
